In [ ]:
from utils.preprocessing import prepare_masks, count_pixels
from utils.features import extract_pixels
from utils.prediction import train_classifier, evaluate_model, predict_tissue
from utils.io import load_metrics
from config import MASKS_PATH, CLASSES, CLASSES_COLORS, TISSUE_MASK
from sklearn.model_selection import train_test_split

In [ ]:
# Step 1: Generate or clean binary masks for each tissue class (e.g. cancer, benign tumor, stroma)
# These masks indicate which pixels in the images belong to which class
prepare_masks(MASKS_PATH, CLASSES, TISSUE_MASK)

# Step 2: Count and collect pixel coordinates for each class from the masks
# Also determine the minimum number of pixels available across all classes
indexes, min_pixels = count_pixels(MASKS_PATH, CLASSES)

# Step 3: Load the spectral image using spectral library
# This returns a memory-mapped image object - <class 'spectral.io.bilfile.BilFile'>
metrics = load_metrics("BR2085d_3_5_metrics_new_MNF20.hdr")

# Step 4: Extract spectral data and corresponding labels from the image
# Only min_pixels per class are extracted to ensure class balance
data, labels = extract_pixels(indexes, min_pixels, metrics)

# Step 5: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

# Step 6: Train a machine learning classifier - Random Forest
classifier = train_classifier(X_train, y_train)
evaluate_model(classifier, X_test, y_test)

Loaded metrics: BR2085d_3_5_metrics_new_MNF20.hdr


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   52.2s


Accuracy: 0.91
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91      7687
           1       0.91      0.90      0.90      7907
           2       0.92      0.92      0.92      7722

    accuracy                           0.91     23316
   macro avg       0.91      0.91      0.91     23316
weighted avg       0.91      0.91      0.91     23316



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.3s


In [ ]:
# Step 8: Use the trained classifier to predict tissue types across the entire image
# Generates an RGB image where each pixel is colored according to its predicted class
predict_tissue(classifier, MASKS_PATH, TISSUE_MASK, metrics, CLASSES_COLORS)

<class 'spectral.io.bipfile.BipFile'>
(1800, 2296, 123)
1
2
3


/home/sonia/.local/lib/python3.10/site-packages/spectral/io/spyfile.py:226: NaNValueWarning: Image data contains NaN values.
  warnings.warn('Image data contains NaN values.', NaNValueWarning)


4


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   15.1s


Prediction done in 24.48s
